In [1]:
from dotenv import load_dotenv
import os 
from langchain_openai import AzureChatOpenAI
from langchain_core.output_parsers import StrOutputParser

load_dotenv('env', override=True)
AZURE_OPENAI_API_KEY = os.getenv('AZURE_OPENAI_API_KEY')
END_POINT=os.getenv('END_POINT')
MODEL_NAME=os.getenv('MODEL_NAME')
print(AZURE_OPENAI_API_KEY[:10])
print(MODEL_NAME)

AZURE_OPENAI_EMB_API_KEY = os.getenv('AZURE_OPENAI_EMB_API_KEY')
EMB_END_POINT=os.getenv('EMB_END_POINT')
EMB_MODEL_NAME=os.getenv('EMB_MODEL_NAME')

os.environ['LANGCHAIN_API_KEY'] = os.getenv('LANGSMITH_API_KEY')
os.environ['LANGCHAIN_ENDPOINT'] = os.getenv('LANGCHAIN_ENDPOINT')
os.environ['LANGCHAIN_TRACING_V2'] = 'true' #true, false
os.environ['LANGCHAIN_PROJECT'] = 'LANG'

if os.getenv('LANGCHAIN_TRACING_V2') == "true":
    print('랭스미스로 추적 중입니다 :', os.getenv('LANGSMITH_API_KEY')[:10])

43b13g4OZS
gpt-4.1-mini
랭스미스로 추적 중입니다 : 


In [5]:
from langchain_openai import AzureChatOpenAI

llm = AzureChatOpenAI(
    api_key=AZURE_OPENAI_API_KEY,
    azure_endpoint=END_POINT,  
    azure_deployment=MODEL_NAME,          
    api_version="2024-12-01-preview",
    temperature=0.1,
)

## 1) open API를 직접 호출하고 분석해주는 체인

In [6]:
from langchain_community.chains.openapi.chain import OpenAPIEndpointChain
from langchain_community.tools.openapi.utils.api_models import APIOperation
from langchain_community.utilities import Requests

#https://petstore3.swagger.io
find_operation = APIOperation.from_openapi_url(
    spec_url="https://petstore3.swagger.io/api/v3/openapi.json", # 재미로 테스트 할 수 있는 펫스토어 오픈 API 문서
    path="/pet/findByStatus",
    method="get",
)
find_operation.base_url = "https://petstore3.swagger.io/api/v3"

find_chain = OpenAPIEndpointChain.from_api_operation(
    find_operation,
    llm=llm,
    requests=Requests(),
    return_intermediate_steps=True,
    verbose=True,
    max_text_length=1000,
)

result = find_chain.invoke({
    "instructions": "status=available 로 검색해서 몇 개 펫의 id만 알려줘. 답변은 한국어로",
    "status": "available"
})
print(result["output"])

Attempting to load an OpenAPI 3.0.4 spec.  This may result in degraded performance. Convert your OpenAPI spec to 3.1.* spec for better support.
/Users/kb/Desktop/AI/kt_ai_academy_202510/.venv/lib/python3.11/site-packages/langchain_community/chains/openapi/chain.py:120: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  _api_arguments = self.api_request_chain.predict_and_parse(




> Entering new OpenAPIEndpointChain chain...


> Entering new APIRequesterChain chain...
Prompt after formatting:
You are a helpful AI Assistant. Please provide JSON arguments to agentFunc() based on the user's instructions.

API_SCHEMA: ```typescript
/* Multiple status values can be provided with comma separated strings. */
type findPetsByStatus = (_: {
/* Status values that need to be considered for filter */
		status: 'available' | 'pending' | 'sold',
}) => any;
```

USER_INSTRUCTIONS: "status=available 로 검색해서 몇 개 펫의 id만 알려줘. 답변은 한국어로"

Your arguments must be plain json provided in a markdown block:

ARGS: ```json
{valid json conforming to API_SCHEMA}
```

Example
-----

ARGS: ```json
{"foo": "bar", "baz": {"qux": "quux"}}
```

The block must be no more than 1 line long, and all arguments must be valid JSON. All string arguments must be wrapped in double quotes.
You MUST strictly comply to the types indicated by the provided schema, including all required args.

If you don't have 

/Users/kb/Desktop/AI/kt_ai_academy_202510/.venv/lib/python3.11/site-packages/langchain_community/chains/openapi/chain.py:155: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  _answer = self.api_response_chain.predict_and_parse(


[{"id":7021978788544032843,"category":{"id":951985472261062482,"name":"1]n2Wjqf^)kmWqywNUw#3(LiXuR~{05t_-f!)XQRhplY|,zSQnAJx2"},"name":"T/","photoUrls":[],"tags":[{"id":7563376621514452005,"name":"T"},{"id":4366091107153711659,"name":"rx}pJNCJU"},{"id":4317627962825250049,"name":"'m-K>|bAe7K'}#kJXW3E93sC>CL|k3O"},{"id":8451547564599435975,"name":"$7BC3a=b/3xA(#Q)A*qT`#B{>l"},{"id":8460391351301096925,"name":");#{5&Z!js|&>5.q>IY3WF?M11MAuL8odwDgxUO/vd]`IU(BBKU;`l"},{"id":1866100844844535119,"name":"X}{{x~LeyX|$#NR)voc:_~1K9o9/erh_`Gpo'<iVkGy[Yey4Q05x"},{"id":7556991464923935645,"name":"`;Bu{ek.1P#8gpq6yD<')cmy]FGAmi3~ivZaIt`!xUH7s{jl4_9NC6q;8^"}],"status":"available"},{"id":3254665021891148501,"category":{"id":951985472261062482,"name":"1]n2Wjqf^)kmWqywNUw#3(LiXuR~{05t_-f!)XQRhplY|,zSQnAJx2"},"name":"T/","photoUrls":[],"tags":[{"id":7563376621514452005,"name":"T"},{"id":4366091107153711659,"name":"rx}pJNCJU"},{"id":4317627962825250049,"name":"'m-K>|bAe7K'}#kJXW3E93sC>CL|k3O"},{"id":8451

In [4]:
from langchain_community.chains.openapi.chain import OpenAPIEndpointChain
from langchain_community.tools.openapi.utils.api_models import APIOperation
from langchain_community.utilities import Requests

pet_info_operation = APIOperation.from_openapi_url(
    spec_url="https://petstore3.swagger.io/api/v3/openapi.json",
    path="/pet/{petId}",
    method="get",
)
pet_info_operation.base_url = "https://petstore3.swagger.io/api/v3"

pet_info_chain = OpenAPIEndpointChain.from_api_operation(
    pet_info_operation,
    llm=llm,
    requests=Requests(),
    return_intermediate_steps=True,
    verbose=True,
    max_text_length=1000,
)

result = pet_info_chain.invoke({
    "instructions": "petId=1로 정보를 한국어로 요약해줘.",
})
print(result["output"])

Attempting to load an OpenAPI 3.0.4 spec.  This may result in degraded performance. Convert your OpenAPI spec to 3.1.* spec for better support.
/Users/kb/Desktop/AI/kt_ai_academy_202510/.venv/lib/python3.11/site-packages/langchain_community/chains/openapi/chain.py:120: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  _api_arguments = self.api_request_chain.predict_and_parse(




> Entering new OpenAPIEndpointChain chain...


> Entering new APIRequesterChain chain...
Prompt after formatting:
You are a helpful AI Assistant. Please provide JSON arguments to agentFunc() based on the user's instructions.

API_SCHEMA: ```typescript
/* Returns a single pet. */
type getPetById = (_: {
/* ID of pet to return */
		petId: number,
}) => any;
```

USER_INSTRUCTIONS: "petId=1로 정보를 한국어로 요약해줘."

Your arguments must be plain json provided in a markdown block:

ARGS: ```json
{valid json conforming to API_SCHEMA}
```

Example
-----

ARGS: ```json
{"foo": "bar", "baz": {"qux": "quux"}}
```

The block must be no more than 1 line long, and all arguments must be valid JSON. All string arguments must be wrapped in double quotes.
You MUST strictly comply to the types indicated by the provided schema, including all required args.

If you don't have sufficient information to call the function due to things like requiring specific uuid's, you can reply with the following message:

Mess

/Users/kb/Desktop/AI/kt_ai_academy_202510/.venv/lib/python3.11/site-packages/langchain_community/chains/openapi/chain.py:155: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  _answer = self.api_response_chain.predict_and_parse(



> Finished chain.
petId가 1인 반려동물은 'Cat 1'이라는 이름의 고양이입니다. 카테고리는 'Cats'이며, 상태는 'available'(구입 가능)입니다. 사진 URL은 'url1'과 'url2'가 있습니다. 태그로는 'tag1'과 'tag2'가 붙어 있습니다.

> Finished chain.
petId가 1인 반려동물은 'Cat 1'이라는 이름의 고양이입니다. 카테고리는 'Cats'이며, 상태는 'available'(구입 가능)입니다. 사진 URL은 'url1'과 'url2'가 있습니다. 태그로는 'tag1'과 'tag2'가 붙어 있습니다.


In [22]:
result = pet_info_chain.invoke({
    "instructions": "petId=1의 사진을 볼 수 있는 url을 알려줘.",
})
print(result["output"])



> Entering new OpenAPIEndpointChain chain...


> Entering new APIRequesterChain chain...
Prompt after formatting:
You are a helpful AI Assistant. Please provide JSON arguments to agentFunc() based on the user's instructions.

API_SCHEMA: ```typescript
/* Returns a single pet. */
type getPetById = (_: {
/* ID of pet to return */
		petId: number,
}) => any;
```

USER_INSTRUCTIONS: "petId=1의 사진을 볼 수 있는 url을 알려줘."

Your arguments must be plain json provided in a markdown block:

ARGS: ```json
{valid json conforming to API_SCHEMA}
```

Example
-----

ARGS: ```json
{"foo": "bar", "baz": {"qux": "quux"}}
```

The block must be no more than 1 line long, and all arguments must be valid JSON. All string arguments must be wrapped in double quotes.
You MUST strictly comply to the types indicated by the provided schema, including all required args.

If you don't have sufficient information to call the function due to things like requiring specific uuid's, you can reply with the following message:

/mnt/c/Users/KaiPharm/dev/ML/uv_lang/.venv/lib/python3.12/site-packages/langchain_community/chains/openapi/chain.py:120: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  _api_arguments = self.api_request_chain.predict_and_parse(



> Finished chain.
{"petId": 1}
{"id":1,"category":{"id":2,"name":"Cats"},"name":"Cat 1","photoUrls":["url1","url2"],"tags":[{"id":1,"name":"tag1"},{"id":2,"name":"tag2"}],"status":"available"}


> Entering new APIResponderChain chain...
Prompt after formatting:
You are a helpful AI assistant trained to answer user queries from API responses.
You attempted to call an API, which resulted in:
API_RESPONSE: {"id":1,"category":{"id":2,"name":"Cats"},"name":"Cat 1","photoUrls":["url1","url2"],"tags":[{"id":1,"name":"tag1"},{"id":2,"name":"tag2"}],"status":"available"}

USER_COMMENT: "petId=1의 사진을 볼 수 있는 url을 알려줘."


If the API_RESPONSE can answer the USER_COMMENT respond with the following markdown json block:
Response: ```json
{"response": "Human-understandable synthesis of the API_RESPONSE"}
```

Otherwise respond with the following markdown json block:
Response Error: ```json
{"response": "What you did and a concise statement of the resulting error. If it can be easily fixed, provide a s

/mnt/c/Users/KaiPharm/dev/ML/uv_lang/.venv/lib/python3.12/site-packages/langchain_community/chains/openapi/chain.py:155: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  _answer = self.api_response_chain.predict_and_parse(



> Finished chain.
petId=1의 사진을 볼 수 있는 URL은 다음과 같습니다: url1, url2

> Finished chain.
petId=1의 사진을 볼 수 있는 URL은 다음과 같습니다: url1, url2


### open api 체인을 사용해 날씨 정보 분석하기

In [3]:
from langchain_community.utilities import Requests
from langchain_community.tools.openapi.utils.api_models import APIOperation
from langchain_community.chains.openapi.chain import OpenAPIEndpointChain

operation = APIOperation.from_openapi_url(
    spec_url="https://raw.githubusercontent.com/open-meteo/open-meteo/main/openapi.yml",
    path="/v1/forecast",
    method="get",
)
operation.base_url = "https://api.open-meteo.com" 

chain = OpenAPIEndpointChain.from_api_operation(
    operation,
    llm=llm,
    requests=Requests(),
    verbose=True,
    return_intermediate_steps=True,
    max_text_length=4000,
)

out_h = chain.invoke({
    "instructions": """시간별 기온을 보고 오늘의 체감기온을 외부활동 기준에서 한국어로 짧게 요약해줘. 
    latitude=37.5665, 
    longitude=126.9780, 
    timezone=Asia/Seoul, 
    forecast_days=1, 
    hourly=temperature_2m,apparent_temperature""",
})

print("\n[시간별 요약]\n", out_h["output"])

Attempting to load an OpenAPI 3.0.0 spec.  This may result in degraded performance. Convert your OpenAPI spec to 3.1.* spec for better support.
/mnt/c/Users/KaiPharm/dev/ML/uv_lang/.venv/lib/python3.12/site-packages/langchain_community/chains/openapi/chain.py:120: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  _api_arguments = self.api_request_chain.predict_and_parse(




> Entering new OpenAPIEndpointChain chain...


> Entering new APIRequesterChain chain...
Prompt after formatting:
You are a helpful AI Assistant. Please provide JSON arguments to agentFunc() based on the user's instructions.

API_SCHEMA: ```typescript
/* 7 day weather variables in hourly and daily resolution for given WGS84 latitude and longitude coordinates. Available worldwide. */
type v1_forecast_get = (_: {
hourly?: Array<DataType.STRING>,

		daily?: Array<DataType.STRING>,
/* WGS84 coordinate */
		latitude: DataType.NUMBER,
/* WGS84 coordinate */
		longitude: DataType.NUMBER,

		current_weather?: DataType.BOOLEAN,

		temperature_unit?: 'celsius' | 'fahrenheit',

		wind_speed_unit?: 'kmh' | 'ms' | 'mph' | 'kn',
/* If format `unixtime` is selected, all time values are returned in UNIX epoch time in seconds. Please not that all time is then in GMT+0! For daily values with unix timestamp, please apply `utc_offset_seconds` again to get the correct date. */
		timeformat?: 'iso8601' | 

/mnt/c/Users/KaiPharm/dev/ML/uv_lang/.venv/lib/python3.12/site-packages/langchain_community/chains/openapi/chain.py:155: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  _answer = self.api_response_chain.predict_and_parse(



> Finished chain.
오늘 서울의 시간별 체감기온은 대체로 14도에서 24도 사이로 쾌적한 편입니다. 아침과 밤에는 약간 선선하지만 낮 동안에는 20도 중반까지 올라 외부활동하기에 매우 좋은 날씨입니다. 가벼운 옷차림으로 야외 활동을 즐기기에 적합합니다.

> Finished chain.

[시간별 요약]
 오늘 서울의 시간별 체감기온은 대체로 14도에서 24도 사이로 쾌적한 편입니다. 아침과 밤에는 약간 선선하지만 낮 동안에는 20도 중반까지 올라 외부활동하기에 매우 좋은 날씨입니다. 가벼운 옷차림으로 야외 활동을 즐기기에 적합합니다.


```python
prompt = PromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, 
                        template='''Translate a math problem into a expression that can be executed using Python\'s numexpr library. 
                        Use the output of running this code to answer the question.\n\nQuestion: ${{Question with math problem.}}\n
                        ```text\n${{single line mathematical expression that solves the problem}}\n```\n...numexpr.evaluate(text)...\n```
                        output\n${{Output of running the code}}\n```\nAnswer: ${{Answer}}\n\nBegin.\n\nQuestion: What is 37593 * 67?\n```
                        text\n37593 * 67\n```\n...numexpr.evaluate("37593 * 67")...\n```output\n2518731\n```\nAnswer: 2518731\n\nQuestion: 37593^(1/5)\n
                        ```text\n37593**(1/5)\n```\n...numexpr.evaluate("37593**(1/5)")...\n
                        ```output\n8.222831614237718\n```\nAnswer: 8.222831614237718\n\nQuestion: {question}\n''')
```

## 2) 문장으로 부터 간단한 수식을 계산해주는 수학 특화 체인

In [7]:
from langchain.chains import LLMMathChain
from langchain.chains.llm_math.prompt import PROMPT

custom_prompt = PROMPT.partial(instructions="질문을 보고 순수한 수식만 작성해. 다른 문장은 절대 출력하지 마.")
math_chain = LLMMathChain.from_llm(llm, prompt=custom_prompt, verbose=True)

print(math_chain.invoke({"question": "1234를 제곱한 값은?"}))
print(math_chain.invoke({"question": "서울-부산 거리 325km를 시속 100km로 가면 몇 시간 걸릴까?"}))
print(math_chain.invoke({"question": "서울-부산을 왕복할 때 325km를 시속 100km로 갔다가 50km로 돌아오면 평균 시속은 몇 km/h 일까?"}))
print(math_chain.invoke({"question": """한 신사가 보석가게에서 70만원짜리 보석을 사고 100만원짜리 수표를 냈다. 
                         때마침 현금이 부족하던 보석가게 주인은 거스름돈을 주기 위해 옆의 빵집에 가서 수표를 현금으로 바꾼 뒤 손님에게 30만원을 거슬러주었다. 
                         그런데 다음날 보석가게 주인이 받았던 수표가 위조수표임이 드러났다. 
                         신사는 이미 보석과 거스름돈을 가지고 간 뒤 였기에 보석가게 주인은 할 수 없이 빵집 주인에게 100만원을 물어주었다. 
                         이때 보석상 주인이 입은 피해액은 얼마인가?"""}))



> Entering new LLMMathChain chain...
1234를 제곱한 값은?```text
1234**2
```
...numexpr.evaluate("1234**2")...

Answer: 1522756
> Finished chain.
{'question': '1234를 제곱한 값은?', 'answer': 'Answer: 1522756'}


> Entering new LLMMathChain chain...
서울-부산 거리 325km를 시속 100km로 가면 몇 시간 걸릴까?```text
325 / 100
```
...numexpr.evaluate("325 / 100")...

Answer: 3.25
> Finished chain.
{'question': '서울-부산 거리 325km를 시속 100km로 가면 몇 시간 걸릴까?', 'answer': 'Answer: 3.25'}


> Entering new LLMMathChain chain...
서울-부산을 왕복할 때 325km를 시속 100km로 갔다가 50km로 돌아오면 평균 시속은 몇 km/h 일까?```text
2 * 325 / (325/100 + 325/50)
```
...numexpr.evaluate("2 * 325 / (325/100 + 325/50)")...

Answer: 66.66666666666667
> Finished chain.
{'question': '서울-부산을 왕복할 때 325km를 시속 100km로 갔다가 50km로 돌아오면 평균 시속은 몇 km/h 일까?', 'answer': 'Answer: 66.66666666666667'}


> Entering new LLMMathChain chain...
한 신사가 보석가게에서 70만원짜리 보석을 사고 100만원짜리 수표를 냈다. 
                         때마침 현금이 부족하던 보석가게 주인은 거스름돈을 주기 위해 옆의 빵집에 가서 수표를 현금으로 바꾼 뒤 손님에게 30만원을 거슬러주었다. 
      

ValueError: unknown format from LLM: Question: 한 신사가 보석가게에서 70만원짜리 보석을 사고 100만원짜리 수표를 냈다. 
                         때마침 현금이 부족하던 보석가게 주인은 거스름돈을 주기 위해 옆의 빵집에 가서 수표를 현금으로 바꾼 뒤 손님에게 30만원을 거슬러주었다. 
                         그런데 다음날 보석가게 주인이 받았던 수표가 위조수표임이 드러났다. 
                         신사는 이미 보석과 거스름돈을 가지고 간 뒤 였기에 보석가게 주인은 할 수 없이 빵집 주인에게 100만원을 물어주었다. 
                         이때 보석상 주인이 입은 피해액은 얼마인가?

```text
700000 + 300000
```
...numexpr.evaluate("700000 + 300000")...